In [1]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

/tmp/ipykernel_82662/4089370055.py:3: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-26 01:07:40 [__init__.py:256] Automatically detected platform cuda.


In [2]:
from unsloth import FastLanguageModel
import torch


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",#"unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    dtype = torch.bfloat16,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.1. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 24.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.18 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [5]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv("data_new.csv")
# df = pd.read_csv("training_data.csv")[["input", "output"]]

# for col in df.columns:
#     df[col] = df[col].str.replace("First National Bank", "ING")
#     df[col] = df[col].str.replace("first national bank", "ING")
#     df[col] = df[col].str.replace("firstnationalbank", "ING")
#     df[col] = df[col].str.replace("FNB", "ING")
#     df[col] = df[col].str.replace("fnb", "ING")

df

,input,output
0,Why is onboarding important for organizations?,Onboarding is important for organizations for ...
1,What are the main benefits of effective onboar...,The main benefits of effective onboarding incl...
2,What should be communicated to a new employee ...,"Before a new employee's first day, you should ..."
3,How should an organization prepare for a new h...,"To prepare for a new hire's arrival, an organi..."
4,What are the key elements of a successful firs...,The key elements of a successful first day for...
...,...,...
135,What should be included in a sample first day ...,# Sample First Day Schedule\n\nThe document pr...
136,How should organizations handle safety and har...,"According to the document, organizations shoul..."
137,What's the most important aspect of preparing ...,"According to the document, ""The most important..."
138,What are some creative ideas for welcoming new...,# Creative Welcome Ideas for New Employees\n\n...


In [6]:
def format_instruction(row):
    return {
        "text": f"<|begin_of_text|><|user|>\n{row['input']}<|end_of_user|>\n<|assistant|>\n{row['output']}<|end_of_assistant|><|end_of_text|>"
    }

formatted_data = [format_instruction(row) for _, row in df.iterrows()]
dataset = Dataset.from_list(formatted_data)


In [9]:
training_args = TrainingArguments(
    output_dir="./llama-3-2-3b-finetuned",
    num_train_epochs=6,
    per_device_train_batch_size=4,
    gradient_checkpointing=True,
    optim="adamw_torch",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=6e-5,
    weight_decay=0.001,
    lr_scheduler_type="cosine",
    warmup_ratio=0.2,
    save_total_limit=3,
)

In [10]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=training_args,
    packing=True,
    dataset_text_field="text"
)

Converting train dataset to ChatML (num_proc=24):   0%|          | 0/140 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=24):   0%|          | 0/140 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=24):   0%|          | 0/140 [00:00<?, ? examples/s]

Packing train dataset (num_proc=24):   0%|          | 0/140 [00:00<?, ? examples/s]

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 27 | Num Epochs = 6 | Total steps = 42
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 20,971,520/8,000,000,000 (0.26% trained)


Step,Training Loss
10,1.856500
20,1.748500
30,1.551600
40,1.505000


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=42, training_loss=1.6531784193856376, metrics={'train_runtime': 138.9812, 'train_samples_per_second': 1.166, 'train_steps_per_second': 0.302, 'total_flos': 7019177601392640.0, 'train_loss': 1.6531784193856376})

In [12]:
prompt = "<|begin_of_text|><|user|>\nWho is responsible for upholding the ethical standards?<|end_of_user|>\n<|assistant|>\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=2048, temperature=0.2)
print(tokenizer.decode(output[0], skip_special_tokens=True))

<|user|>
Who is responsible for upholding the ethical standards?<|end_of_user|>
<|assistant|>
The responsibility for upholding ethical standards lies with everyone in the organization, from the CEO to the front-line employees. All employees are expected to adhere to the organization's code of conduct and report any violations they observe.<|end_of_assistant|>


In [13]:
model_path = "./llama-3-2-8b-finetuned-final"
trainer.save_model(model_path)

In [20]:
import torch
from unsloth import FastLanguageModel
from transformers import TextIteratorStreamer
from threading import Thread
import time
import sys


def load_model(model_path):
    """
    Load the fine-tuned model with 4-bit quantization
    """
    print(f"Loading model from {model_path}...")
    
    try:
        # Load the fine-tuned model and tokenizer
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=4096,
            load_in_4bit=True,
            attn_implementation="flash_attention_2",  # Use flash attention for better performance if available
        )
    except Exception as e:
        print(f"Error with flash_attention_2: {e}")
        print("Trying without flash attention...")
        # Fallback to standard attention
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_path,
            max_seq_length=4096,
            load_in_4bit=True,
        )
    
    # Configure the generation parameters
    model.config.do_sample = True  # Enable sampling
    model.config.temperature = 0.1  # Temperature for randomness (lower = more deterministic)
    model.config.top_p = 0.9  # Nucleus sampling parameter
    
    print("Model loaded successfully!")
    
    return model, tokenizer

def format_prompt(user_input):
    """Format the prompt according to Llama 3.2 chat template"""
    return f"<|begin_of_text|><|user|>\n{user_input}<|end_of_user|>\n<|assistant|>\n"

def generate_with_streaming(model, tokenizer, prompt, max_new_tokens=512):
    """
    Generate text with token-by-token streaming
    """
    # Initialize the streamer
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True, skip_prompt=True)
    
    # Tokenize the input and handle it properly
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)
    
    # Start generation in a separate thread
    generation_kwargs = dict(
        input_ids=input_ids,
        attention_mask=attention_mask, 
        max_new_tokens=max_new_tokens,
        streamer=streamer,
        temperature=0.7,
        top_p=0.9,
    )
    
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()
    
    # Stream the output token by token
    generated_text = ""
    print("\nAssistant: ", end="", flush=True)
    for new_text in streamer:
        print(new_text, end="", flush=True)
        generated_text += new_text
        time.sleep(0.01)  # Small delay to simulate token-by-token generation
    print("\n")
    
    return generated_text


model_path = "./llama-3-2-8b-finetuned-final"  # Change this to your model path
    
    # Load the model
model, tokenizer = load_model(model_path)

print("=" * 50)
print("Fine-tuned Llama 3.2 3B Inference")
print("Enter your prompts below. Type 'exit' to quit.")
print("=" * 50)

# Interactive loop
while True:
    user_input = input("\nUser: ")
    
    if user_input.lower() in ["exit", "quit", "q"]:
        print("Exiting...")
        break
    
    # Format the prompt
    prompt = format_prompt(user_input)
    
    # Generate and stream response
    generate_with_streaming(model, tokenizer, prompt)

Loading model from ./llama-3-2-8b-finetuned-final...
==((====))==  Unsloth 2025.3.18: Fast Llama patching. Transformers: 4.50.1. vLLM: 0.8.1.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 24.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully!
Fine-tuned Llama 3.2 3B Inference
Enter your prompts below. Type 'exit' to quit.

Assistant: At the heart of our company is a set of core values that guide our decision-making, interactions with others, and overall culture. These values are: Integrity, Excellence, Innovation, Collaboration, Respect, Accountability, and Diversity.<|end_of_assistant|>


Assistant: If you see unethical behavior, you should first try to resolve the issue d

KeyboardInterrupt: 

In [20]:
dataset

[[{'content': "The Employee Handbook is designed to provide clear guidelines regarding ING's policies, procedures, and practices; outline the benefits available to eligible employees; communicate expectations for employee conduct and performance; ensure compliance with banking regulations and employment laws; and foster a positive and productive work environment. While the handbook contains a great deal of information, it cannot anticipate every situation or answer every question about employment at ING. The policies described are not conditions of employment, and the language is not intended to create a contract between ING and its employees.",
   'role': 'user'},
  {'content': 'What is the purpose of the ING Employee Handbook?',
   'role': 'assistant'}],
 [{'content': "ING's core values are: 1) Integrity: Conducting business ethically and honestly, always doing what is right for customers, employees, and communities; 2) Excellence: Striving for exceptional performance in everything, 

In [22]:
formatted_data = []

for conversation in dataset:
    user_message = conversation[0]["content"]
    assistant_message = conversation[1]["content"]

    formatted_entry = {
        "user_message": user_message,
        "assistant_message": assistant_message,
        "conversations": [
            {"role": "user", "content": assistant_message},
            {"role": "assistant", "content": user_message}
        ]
    }
    
    formatted_data.append(formatted_entry)

dataset_hf = Dataset.from_pandas(pd.DataFrame(formatted_data))

In [23]:
dataset_hf[5]

{'user_message': "The first 90 days of employment at ING are considered a probationary period. During this time, employees have the opportunity to evaluate the Bank as a place to work, and management has the opportunity to evaluate the employee's job performance. New employees will receive training and guidance from their supervisor, and performance evaluations may be conducted at the end of the probationary period. Completion of the probationary period does not guarantee continued employment and does not change the at-will employment relationship. Either the employee or the Bank may end the employment relationship at any time during or after the probationary period, with or without cause or advance notice.",
 'assistant_message': 'What is the probationary period for new employees?',
 'conversations': [{'content': 'What is the probationary period for new employees?',
   'role': 'user'},
  {'content': "The first 90 days of employment at ING are considered a probationary period. During t

In [24]:
dataset_hf["assistant_message"]

['What is the purpose of the ING Employee Handbook?',
 'What are the core values of ING?',
 "What is ING's Equal Employment Opportunity policy?",
 "Explain ING's employment-at-will policy.",
 'What are the different employee classifications at ING?',
 'What is the probationary period for new employees?',
 "What is ING's policy on employment of relatives?",
 "What are the bank's expectations regarding confidentiality and non-disclosure?",
 "What is included in ING's Code of Ethics?",
 "What is ING's dress code policy?",
 'What are the attendance and punctuality expectations at ING?',
 "What are ING's customer service standards?",
 'How does ING define conflicts of interest?',
 "What is ING's policy on gifts and gratuities?",
 'What are the guidelines for outside employment at ING?',
 "What is ING's social media policy?",
 "What are ING's privacy and confidentiality requirements?",
 "What constitutes sexual harassment according to ING's policy?",
 'How should employees report harassment 

In [ ]:
dataset_hf_final = dataset_hf.map(
    lambda examples: {
        "text": [
            f"{}"
            for conv in examples["conversations"]
        ]
    },
    batched=True
)

Map:   0%|          | 0/481 [00:00<?, ? examples/s]

In [19]:
dataset_hf_final[5]["text"]

[128000,
 128006,
 9125,
 128007,
 271,
 38766,
 1303,
 33025,
 2696,
 25,
 6790,
 220,
 2366,
 18,
 198,
 15724,
 2696,
 25,
 220,
 1627,
 5887,
 220,
 2366,
 19,
 271,
 128009,
 128006,
 882,
 128007,
 271,
 3923,
 374,
 279,
 47027,
 661,
 4261,
 369,
 502,
 8420,
 30,
 128009,
 128006,
 78191,
 128007,
 271,
 791,
 1176,
 220,
 1954,
 2919,
 315,
 14740,
 520,
 2006,
 38,
 527,
 6646,
 264,
 47027,
 661,
 4261,
 13,
 12220,
 420,
 892,
 11,
 8420,
 617,
 279,
 6776,
 311,
 15806,
 279,
 8715,
 439,
 264,
 2035,
 311,
 990,
 11,
 323,
 6373,
 706,
 279,
 6776,
 311,
 15806,
 279,
 9548,
 596,
 2683,
 5178,
 13,
 1561,
 8420,
 690,
 5371,
 4967,
 323,
 19351,
 505,
 872,
 38419,
 11,
 323,
 5178,
 56181,
 1253,
 387,
 13375,
 520,
 279,
 842,
 315,
 279,
 47027,
 661,
 4261,
 13,
 57350,
 315,
 279,
 47027,
 661,
 4261,
 1587,
 539,
 15803,
 8738,
 14740,
 323,
 1587,
 539,
 2349,
 279,
 520,
 2695,
 484,
 14740,
 5133,
 13,
 21663,
 279,
 9548,
 477,
 279,
 8715,
 1253,
 842,
 279,


In [ ]:


trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_hf,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 8,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

KeyError: 'text'

In [8]:
dataset

,instruction,input,output
0,NaN,What is the purpose of the First National Bank...,The Employee Handbook is designed to provide c...
1,NaN,What are the core values of First National Bank?,First National Bank's core values are: 1) Inte...
2,NaN,What is First National Bank's Equal Employment...,First National Bank is committed to providing ...
3,NaN,Explain First National Bank's employment-at-wi...,Employment with First National Bank is volunta...
4,NaN,What are the different employee classification...,First National Bank classifies its employees a...
...,...,...,...
476,NaN,What should I do if I see another employee vio...,Employees are expected to report suspected vio...
477,NaN,Are visible tattoos allowed at work?,Visible tattoos should be minimal and non-offe...
478,NaN,Can I use colored hair dye?,Unnaturally colored hair is generally not perm...
479,NaN,What types of facial piercings are allowed?,Facial jewelry other than earrings should be m...
